In [1]:
from STvAR_v3 import *
from STvARMetricObjects import *
from STvARFiniteDifference import *
from STvARAdvection import *
from STvARSymbolParsing import *
from STvARIndexing import*

In [2]:
DiffOrder = 4
DiffOrderKO = DiffOrder + 2
KOsigma = 0.3
dim = 3

In [3]:
gridvars = []
vartypes = []

chi = stvar('chi',  gridvar = True, vartype = 'state_fab')
gamtilde_LL = stvarrank2('gamtilde_LL',  gridvar = True, vartype = 'state_fab', sym = 'sym01')
Khat = stvar('Khat', gridvar = True, vartype = 'state_fab')
Atilde_LL = stvarrank2('Atilde_LL', gridvar = True, vartype = 'state_fab', sym = 'sym01')
theta = stvar('theta', gridvar = True, vartype = 'state_fab')
Gamtilde_U = stvarrank1('Gamtilde_U', gridvar = True, vartype ='state_fab')

alpha = stvar('alpha', gridvar = True, vartype = 'state_fab')
beta_U = stvarrank1('beta_U', gridvar = True, vartype = 'state_fab')

In [4]:
declareStates = ""
declareStates += chi.symb2isymb()
declareStates += gamtilde_LL.symb2isymb()
declareStates += Khat.symb2isymb()
declareStates += Atilde_LL.symb2isymb()
declareStates += theta.symb2isymb()
declareStates += Gamtilde_U.symb2isymb()

declareStates += alpha.symb2isymb()
declareStates += beta_U.symb2isymb()

In [5]:
declareStates = declareStates.replace("state_fabIdx", "Idx")

In [6]:
dDchi_L = chi.diff('dD', Accuracy = DiffOrder)
dDDchi_LL = chi.diff('dDD', Accuracy = DiffOrder)
dDgamtilde_LLL = gamtilde_LL.diff('dD', Accuracy = DiffOrder)
dDDgamtilde_LLLL = gamtilde_LL.diff('dDD', Accuracy = DiffOrder)
dDKhat_L = Khat.diff('dD', Accuracy = DiffOrder)
dDAtilde_LLL = Atilde_LL.diff('dD', Accuracy = DiffOrder)
dDtheta_L = theta.diff('dD', Accuracy = DiffOrder)

In [7]:
declareDerivatives = ""
declareDerivatives += dDchi_L.symb2isymb()
declareDerivatives += dDDchi_LL.symb2isymb()
declareDerivatives += dDgamtilde_LLL.symb2isymb()
declareDerivatives += dDDgamtilde_LLLL.symb2isymb()
declareDerivatives += dDKhat_L.symb2isymb()
declareDerivatives += dDAtilde_LLL.symb2isymb()
declareDerivatives += dDtheta_L.symb2isymb()
declareDerivatives = declareDerivatives.replace("state_fabIdx", "Idx")

In [8]:
metricObjects = ""

In [9]:
gamtilde_UU = inversemetric(gamtilde_LL)

In [10]:
metricObjects += gamtilde_UU.symb2expr()

In [11]:
Gamtilde_LLL = ChristoffelLLL(gamtilde_LL)
Gamtilde_ULL = ChristoffelULL(gamtilde_LL)

In [12]:
metricObjects += Gamtilde_LLL.symb2expr()
metricObjects += Gamtilde_ULL.symb2expr()

In [13]:
gam_LL = stvarrank2('gam_LL')
gam_LL.expr = (chi.symb**(-1))*gamtilde_LL.symb

gam_UU = stvarrank2('invgam_UU')
gam_UU.expr = chi.symb*gamtilde_UU.symb

In [14]:
Gam_ULL = ConformalChristoffelULL(chi, gamtilde_LL)
Gam_LLL = ConformalChristoffelLLL(chi, gamtilde_LL)

In [15]:
metricObjects += gam_LL.symb2expr()
metricObjects += gam_UU.symb2expr()
metricObjects += Gam_ULL.symb2expr()
metricObjects += Gam_LLL.symb2expr()

In [16]:
dDDgam_LLLL= stvarrank4('dDDgam_LLLL', sym = 'sym01sym23')
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                dDDgam_LLLL.expr[i][j][k][l] += -dDDchi_LL.symb[k][l]/(chi.symb**2)*gamtilde_LL.symb[i][j]
                dDDgam_LLLL.expr[i][j][k][l] += 2*dDchi_L.symb[k]*dDchi_L.symb[l]/(chi.symb**3)*gamtilde_LL.symb[i][j]
                dDDgam_LLLL.expr[i][j][k][l] += -dDchi_L.symb[k]/(chi.symb**2)*dDgamtilde_LLL.symb[i][j][l]-dDchi_L.symb[l]/(chi.symb**2)*dDgamtilde_LLL.symb[i][j][k]
                dDDgam_LLLL.expr[i][j][k][l] += dDDgamtilde_LLLL.symb[i][j][k][l]/chi.symb
                
                

In [17]:
metricObjects += dDDgam_LLLL.symb2expr()

In [18]:
Riem_LLLL = stvarrank4('Riem_LLLL', sym = 'asym01asym23')
for i in range(3):
    for k in range(3):
        for l in range(3):
            for m in range(3):
                Riem_LLLL.expr[i][k][l][m] = 1/2*(dDDgam_LLLL.symb[i][m][k][l]+dDDgam_LLLL.symb[k][l][i][m]-dDDgam_LLLL.symb[i][l][k][m]-dDDgam_LLLL.symb[k][m][i][l])
                for p in range(3):
                    Riem_LLLL.expr[i][k][l][m] += Gam_LLL.symb[p][k][l]*Gam_ULL.symb[p][i][m]-Gam_LLL.symb[p][k][m]*Gam_ULL.symb[p][i][l]

In [19]:
metricObjects += Riem_LLLL.symb2expr()

In [20]:
Ricci_LL = stvarrank2('Ricci_LL', sym = 'sym01')
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):
                    Ricci_LL.expr[j][l] += gam_UU.symb[i][m]*Riem_LLLL.symb[i][j][m][l]

In [21]:
metricObjects += Ricci_LL.symb2expr()

In [22]:
K_LL = stvarrank2('K_LL', sym = 'sym01')
for i in range(3):
    for j in range(3):
        K_LL.expr[i][j] += (chi.symb**(-1))*(Atilde_LL.symb[i][j]+1/3*gamtilde_LL.symb[i][j]*(Khat.symb+2*theta.symb))

In [23]:
metricObjects += K_LL.symb2expr()

In [24]:
Ksclr = stvar('Ksclr')
Ksclr.expr = Khat.symb + 2*theta.symb

In [25]:
metricObjects += Ksclr.symb2expr()

In [26]:
detgam = detmetric(gam_LL)

In [27]:
metricObjects += detgam.symb2expr()

In [28]:
tetradObjects = ""

In [29]:
x = stvar('x')
x.expr = 'x0'
y = stvar('y')
y.expr = 'x1'
z = stvar('z')
z.expr = 'x2'

rcoord = stvar('rcoord')
rcoord.expr = sp.sqrt(x.symb**2 + y.symb**2 + z.symb**2)

In [30]:
tetradObjects += x.symb2expr() + y.symb2expr()+z.symb2expr() + rcoord.symb2expr()

In [31]:
v0_U = stvarrank1('v0_U')
v0_U.expr = [-y.symb,x.symb,0]
v1_U = stvarrank1('v1_U')
v1_U.expr = [x.symb,y.symb,z.symb]

In [38]:
v2_U = stvarrank1('v2_U')
for a in range(3):
    for b in range(3):
        for c in range(3):
            for d in range(3):
                v2_U.expr[a] += sp.sqrt(detgam.symb)*gam_UU.symb[a][d]*sp.Eijk(d,b,c)*v0_U.symb[b]*v1_U.symb[c]

In [40]:
print(v2_U.symb2expr())

        amrex::Real v2_U_0 = std::sqrt(detgam)*invgam_UU_00*v0_U_1*v1_U_2 - std::sqrt(detgam)*invgam_UU_00*v0_U_2*v1_U_1 - std::sqrt(detgam)*invgam_UU_01*v0_U_0*v1_U_2 + std::sqrt(detgam)*invgam_UU_01*v0_U_2*v1_U_0 + std::sqrt(detgam)*invgam_UU_02*v0_U_0*v1_U_1 - std::sqrt(detgam)*invgam_UU_02*v0_U_1*v1_U_0;
        amrex::Real v2_U_1 = std::sqrt(detgam)*invgam_UU_10*v0_U_1*v1_U_2 - std::sqrt(detgam)*invgam_UU_10*v0_U_2*v1_U_1 - std::sqrt(detgam)*invgam_UU_11*v0_U_0*v1_U_2 + std::sqrt(detgam)*invgam_UU_11*v0_U_2*v1_U_0 + std::sqrt(detgam)*invgam_UU_12*v0_U_0*v1_U_1 - std::sqrt(detgam)*invgam_UU_12*v0_U_1*v1_U_0;
        amrex::Real v2_U_2 = std::sqrt(detgam)*invgam_UU_20*v0_U_1*v1_U_2 - std::sqrt(detgam)*invgam_UU_20*v0_U_2*v1_U_1 - std::sqrt(detgam)*invgam_UU_21*v0_U_0*v1_U_2 + std::sqrt(detgam)*invgam_UU_21*v0_U_2*v1_U_0 + std::sqrt(detgam)*invgam_UU_22*v0_U_0*v1_U_1 - std::sqrt(detgam)*invgam_UU_22*v0_U_1*v1_U_0;




In [33]:
tetradObjects += v0_U.symb2expr()+v1_U.symb2expr()+v2_U.symb2expr()

In [34]:
omega00 = stvar('omega00')
for a in range(3):
    for b in range(3):
        omega00.expr += v0_U.symb[a]*v0_U.symb[b]*gam_LL.symb[a][b]

In [35]:
tetradObjects += omega00.symb2expr()

In [36]:
e0_U = stvarrank1('e0_U')
for a in range(3):
    e0_U.expr[a] = v0_U.symb[a]/omega00.symb

In [37]:
tetradObjects += e0_U.symb2expr()

In [38]:
omega01 = stvar('omega01')
for a in range(3):
    for b in range(3):
        omega01.expr += e0_U.symb[a]*v1_U.symb[b]*gam_LL.symb[a][b]

vp1_U = stvarrank1('vp1_U')
for a in range(3):
    vp1_U.expr[a] = v1_U.symb[a]-omega01.symb*e0_U.symb[a]

omega11 = stvar('omega11')
for a in range(3):
    for b in range(3):
        omega11.expr += vp1_U.symb[a]*vp1_U.symb[b]*gam_LL.symb[a][b]
        
e1_U = stvarrank1('e1_U')
for a in range(3):
    e1_U.expr[a] = vp1_U.symb[a]/omega11.symb

In [39]:
tetradObjects += omega01.symb2expr() + vp1_U.symb2expr() + omega11.symb2expr() + e1_U.symb2expr()

In [40]:
omega02 = stvar('omega02')
omega12 = stvar('omega12')
for a in range(3):
    for b in range(3):
        omega02.expr += e0_U.symb[a]*v2_U.symb[b]*gam_LL.symb[a][b]
        omega12.expr += e1_U.symb[a]*v2_U.symb[b]*gam_LL.symb[a][b]

vp2_U = stvarrank1('vp2_U')
for a in range(3):
    vp2_U.expr[a] = v2_U.symb[a]-e0_U.symb[a]*omega02.symb-e1_U.symb[a]*omega12.symb
    
omega22 = stvar('omega22')
for a in range(3):
    for b in range(3):
        omega22.expr += vp2_U.symb[a]*vp2_U.symb[b]*gam_LL.symb[a][b]+(sp.Abs(vp2_U.symb[a]*vp2_U.symb[b]*gam_LL.symb[a][b])<0.00001)
        
e2_U = stvarrank1('e2_U')
for a in range(3):
    e2_U.expr[a] = vp2_U.symb[a]/omega22.symb

In [41]:
tetradObjects += omega02.symb2expr() + omega12.symb2expr() + vp2_U.symb2expr() + omega22.symb2expr()+e2_U.symb2expr()
#tetradObjects += """
#        amrex::Real e2_U_0 = vp2_U_0;
#        amrex::Real e2_U_1 = vp2_U_1;
#        amrex::Real e2_U_2 = vp2_U_2;
#        
#        if (omega22 != 0)
#        {
#            e2_U_0 = vp2_U_0/omega22;
#            e2_U_1 = vp2_U_1/omega22;
#            e2_U_2 = vp2_U_2/omega22;
#        }
#"""

In [42]:
u_U = stvarrank1('u_U', dim = 4)
r_U = stvarrank1('r_U', dim = 4)
theta_U = stvarrank1('theta_U', dim = 4)
phi_U = stvarrank1('phi_U', dim = 4)
u_U.expr[3] = alpha.symb**(-1)
r_U.expr[3] = 0
theta_U.expr[3] = 0
phi_U.expr[3] = 0
for i in range(3):
    u_U.expr[i] = - beta_U.symb[i]/alpha.symb
    r_U.expr[i] = e1_U.symb[i]
    theta_U.expr[i] = e2_U.symb[i]
    phi_U.expr[i] = e0_U.symb[i]

In [43]:
tetradObjects += u_U.symb2expr() + r_U.symb2expr() + theta_U.symb2expr() + phi_U.symb2expr()

In [44]:
l_U = stvarrank1('l_U', dim = 4)
l_U.expr =  1/sp.sqrt(2)*(u_U.symb+r_U.symb)

n_U = stvarrank1('n_U', dim = 4)
n_U.expr =  1/sp.sqrt(2)*(u_U.symb-r_U.symb)

mre_U = stvarrank1('mre_U', dim = 4)
mre_U.expr =  1/sp.sqrt(2)*theta_U.symb

mim_U = stvarrank1('mim_U', dim = 4)
mim_U.expr =  1/sp.sqrt(2)*phi_U.symb

In [45]:
tetradObjects += l_U.symb2expr() + n_U.symb2expr() + mre_U.symb2expr() + mim_U.symb2expr()

In [46]:
nmnmre_UUUU = stvarrank4('nmnmre_UUUU', sym = 'sym13', dim = 4)
nmnmim_UUUU = stvarrank4('nmnmim_UUUU', sym = 'sym13', dim = 4)

In [47]:
for a in range(4):
    for b in range(4):
        for c in range(4):
            for d in range(4):
                nmnmre_UUUU.expr[a][b][c][d] += n_U.symb[a]*mre_U.symb[b]*n_U.symb[c]*mre_U.symb[d]-n_U.symb[a]*mim_U.symb[b]*n_U.symb[c]*mim_U.symb[d]
                nmnmim_UUUU.expr[a][b][c][d] += -n_U.symb[a]*mre_U.symb[b]*n_U.symb[c]*mim_U.symb[d]-n_U.symb[a]*mim_U.symb[b]*n_U.symb[c]*mre_U.symb[d]

In [48]:
tetradObjects += nmnmre_UUUU.symb2expr() +nmnmim_UUUU.symb2expr()

In [49]:
rank4Part_LLLL = stvarrank4('rank4Part_LLLL', sym = 'asym01asym23')
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                rank4Part_LLLL.expr[i][j][k][l] = Riem_LLLL.symb[i][j][k][l] + K_LL.symb[i][k]*K_LL.symb[l][j] - K_LL.symb[i][l]*K_LL.symb[k][j]

In [50]:
tetradObjects += rank4Part_LLLL.symb2expr()

In [51]:
dDK_LLL = stvarrank3('dDK_LLL', sym = 'sym01')
for i in range(3):
    for j in range(3):
        for k in range(3):
            dDK_LLL.expr[i][j][k] += -dDchi_L.symb[k]*K_LL.symb[i][j] + dDAtilde_LLL.symb[i][j][k]+1/3*dDgamtilde_LLL.symb[i][j][k]*Ksclr.symb+1/3*gamtilde_LL.symb[i][j]*(dDKhat_L.symb[k] + 2*dDtheta_L.symb[k])
            dDK_LLL.expr[i][j][k] /= chi.symb

In [52]:
tetradObjects += dDK_LLL.symb2expr()

In [53]:
rank3Part_LLL = stvarrank3('rank3Part_LLL', sym = 'asym12')
for j in range(3):
    for k in range(3):
        for l in range(3):
            rank3Part_LLL.expr[j][k][l] += -4*(dDK_LLL.symb[j][k][l] - dDK_LLL.symb[j][l][k]) 
            for p in range(3):
                rank3Part_LLL.expr[j][k][l] += -4*(Gam_ULL.symb[p][j][k]*K_LL.symb[l][p]-Gam_ULL.symb[p][j][l]*K_LL.symb[k][p])

In [54]:
tetradObjects += rank3Part_LLL.symb2expr()

In [55]:
rank2Part_LL = stvarrank2('rank2Part_LL',sym = 'sym01')
for j in range(3):
    for l in range(3):
        rank2Part_LL.expr[j][l] += 4*(Ricci_LL.symb[j][l]+Ksclr.symb*K_LL.symb[j][l])
        for m in range(3):
            for l in range(3):
                rank2Part_LL.expr[j][l] += -4*K_LL.symb[j][p]*gam_UU.symb[p][m]*K_LL.symb[m][l]

In [56]:
tetradObjects += rank2Part_LL.symb2expr()

In [57]:
Psi4re = stvar('Psi4re', gridvar = True, vartype = 'Diag', varprefix = 'diag')
Psi4im = stvar('Psi4im', gridvar = True, vartype = 'Diag', varprefix = 'diag')
for i in range(3):
    for j in range(3):
        Psi4re.expr += 1/4*rank2Part_LL.symb[i][j]*(nmnmre_UUUU.symb[3][i][3][j]-nmnmre_UUUU.symb[j][3][3][i]-nmnmre_UUUU.symb[3][i][j][3]+nmnmre_UUUU.symb[i][3][j][3])
        Psi4im.expr += 1/4*rank2Part_LL.symb[i][j]*(nmnmim_UUUU.symb[3][i][3][j]-nmnmim_UUUU.symb[j][3][3][i]-nmnmim_UUUU.symb[3][i][j][3]+nmnmim_UUUU.symb[i][3][j][3])
        for k in range(3):
            Psi4re.expr += 1/2*rank3Part_LLL.symb[i][j][k]*(nmnmre_UUUU.symb[3][i][j][k]-nmnmre_UUUU.symb[i][3][j][k])
            Psi4im.expr += 1/2*rank3Part_LLL.symb[i][j][k]*(nmnmim_UUUU.symb[3][i][j][k]-nmnmim_UUUU.symb[i][3][j][k])
            for l in range(3):
                Psi4re.expr += rank4Part_LLLL.symb[i][j][k][l]*nmnmre_UUUU.symb[i][j][k][l]
                Psi4im.expr += rank4Part_LLLL.symb[i][j][k][l]*nmnmim_UUUU.symb[i][j][k][l]
                
Psi4re.expr *= (rcoord.symb > 5)
Psi4im.expr *= (rcoord.symb > 5)

In [58]:
tetradObjects += Psi4re.setisymb()
tetradObjects += Psi4im.setisymb()

In [59]:
FullString = declareStates + declareDerivatives + metricObjects+tetradObjects

In [60]:
FullString = FullString.replace('state_fabIdx', 'Idx')
FullString = FullString.replace('DiagIdx', 'Diag')

In [61]:
print(FullString)

        amrex::Real chi = state_fab(i, j, k, Idx::chi);

        amrex::Real gamtilde_LL_00 = state_fab(i, j, k, Idx::gamtilde_LL_00);
        amrex::Real gamtilde_LL_01 = state_fab(i, j, k, Idx::gamtilde_LL_01);
        amrex::Real gamtilde_LL_02 = state_fab(i, j, k, Idx::gamtilde_LL_02);
        amrex::Real gamtilde_LL_11 = state_fab(i, j, k, Idx::gamtilde_LL_11);
        amrex::Real gamtilde_LL_12 = state_fab(i, j, k, Idx::gamtilde_LL_12);
        amrex::Real gamtilde_LL_22 = state_fab(i, j, k, Idx::gamtilde_LL_22);

        amrex::Real Khat = state_fab(i, j, k, Idx::Khat);

        amrex::Real Atilde_LL_00 = state_fab(i, j, k, Idx::Atilde_LL_00);
        amrex::Real Atilde_LL_01 = state_fab(i, j, k, Idx::Atilde_LL_01);
        amrex::Real Atilde_LL_02 = state_fab(i, j, k, Idx::Atilde_LL_02);
        amrex::Real Atilde_LL_11 = state_fab(i, j, k, Idx::Atilde_LL_11);
        amrex::Real Atilde_LL_12 = state_fab(i, j, k, Idx::Atilde_LL_12);
        amrex::Real Atilde_LL_22 = state_fab(